### Setup

In [1]:
!pip3 install gensim --quiet
!pip3 install nltk --quiet
!pip3 install spacy --quiet
!pip3 install pandas --quiet
!pip3 install pyLDAvis --quiet
!pip3 install matplotlib --quiet
!python3 -m spacy download en

Looking in indexes: http://10.100.1.91/packages/stable
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/nfs/home/ramtin/environments/experiments/lib/python3.5/site-packages/en_core_web_sm
-->
/nfs/home/ramtin/environments/experiments/lib/python3.5/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [2]:
import re
import random
import numpy as np
import pandas as pd
from pprint import pprint

# nltk
import nltk; nltk.download('stopwords')
from nltk.corpus import stopwords

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     /nfs/home/ramtin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress


In [3]:
# NLTK Stop words
stop_words = stopwords.words('english')
# if we need more terms to be added here
stop_words.extend(['section', 'pt'])

## Loading mimic3 noteevents 

In [10]:
# Load data from gzip CSV 
n = 2083180 # number of records in file
s = 200000 # desired sample size
data_file = '/shared2/datasets/mimic3/NOTEEVENTS.csv.gz'
# start the range with 1 to keep the headers
skip = sorted(random.sample(range(1, n), n-s))


df = pd.read_csv(data_file,
                 skiprows=skip,
                 compression='gzip',
                 usecols=["ROW_ID", "CATEGORY", "TEXT"],
                 error_bad_lines=False)

In [11]:
### data loading 
df

,ROW_ID,CATEGORY,TEXT
0,178,Discharge summary,Admission Date: [**2162-3-3**] D...
1,182,Discharge summary,Admission Date: [**2118-8-10**] ...
2,243,Discharge summary,Admission Date: [**2186-6-7**] Discharge ...
3,189,Discharge summary,Admission Date: [**2147-6-24**] Discharge...
4,190,Discharge summary,Admission Date: [**2102-2-19**] Dischar...
5,222,Discharge summary,Admission Date: [**2121-3-31**] ...
6,20,Discharge summary,Admission Date: [**2178-11-15**] ...
7,46,Discharge summary,Admission Date: [**2112-6-6**] Discharg...
8,53,Discharge summary,Admission Date: [**2101-11-1**] ...
9,103,Discharge summary,Admission Date: [**2191-4-24**] ...


In [12]:
## Normalizations
# Convert to list
data = df["TEXT"].values.tolist()
# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]
# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]
# slice an example from output
pprint(data[:1])

['Admission Date: [**2162-3-3**] Discharge Date: [**2162-3-25**] Date of '
 'Birth: [**2080-1-4**] Sex: M Service: MEDICINE Allergies: Patient recorded '
 'as having No Known Allergies to Drugs Attending:[**First Name3 (LF) 1828**] '
 'Chief Complaint: Mr. [**Known lastname 1829**] was seen at [**Hospital1 '
 '18**] after a mechanical fall from a height of 10 feet. CT scan noted '
 'unstable fracture of C6-7 & posterior elements. Major Surgical or Invasive '
 'Procedure: 1. Anterior cervical osteotomy, C6-C7, with decompression and '
 'excision of ossification of the posterior longitudinal ligament. 2. Anterior '
 'cervical deformity correction. 3. Interbody reconstruction. 4. Anterior '
 'cervical fusion, C5-C6-C7. 5. Plate instrumentation, C5-C6-C7. 6. Cervical '
 'laminectomy C6-C7, T1. 7. Posterior cervical arthrodesis C4-T1. 8. Cervical '
 'instrumentation C4-T1. 9. Arthrodesis augmentation with autograft, allograft '
 'and demineralized bone matrix. History of Present Illness: Mr

In [13]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))
print(data_words[:1])

[['admission', 'date', 'discharge', 'date', 'date', 'of', 'birth', 'sex', 'service', 'medicine', 'allergies', 'patient', 'recorded', 'as', 'having', 'no', 'known', 'allergies', 'to', 'drugs', 'attending', 'first', 'name', 'lf', 'chief', 'complaint', 'mr', 'known', 'lastname', 'was', 'seen', 'at', 'hospital', 'after', 'mechanical', 'fall', 'from', 'height', 'of', 'feet', 'ct', 'scan', 'noted', 'unstable', 'fracture', 'of', 'posterior', 'elements', 'major', 'surgical', 'or', 'invasive', 'procedure', 'anterior', 'cervical', 'osteotomy', 'with', 'decompression', 'and', 'excision', 'of', 'ossification', 'of', 'the', 'posterior', 'longitudinal', 'ligament', 'anterior', 'cervical', 'deformity', 'correction', 'interbody', 'reconstruction', 'anterior', 'cervical', 'fusion', 'plate', 'instrumentation', 'cervical', 'laminectomy', 'posterior', 'cervical', 'arthrodesis', 'cervical', 'instrumentation', 'arthrodesis', 'augmentation', 'with', 'autograft', 'allograft', 'and', 'demineralized', 'bone', '

In [14]:
# Store data_words to file 
import pickle 
with open('200K_sample_data_words.pickle', 'wb') as outp:
    pickle.dump(data_words, outp)

In [15]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['admission', 'date', 'discharge', 'date', 'date', 'of', 'birth_sex_service_medicine', 'allergies', 'patient', 'recorded', 'as', 'having', 'no', 'known', 'allergies', 'to', 'drugs_attending', 'first', 'name', 'lf', 'chief_complaint', 'mr', 'known', 'lastname', 'was', 'seen', 'at', 'hospital', 'after', 'mechanical', 'fall', 'from', 'height', 'of', 'feet', 'ct', 'scan', 'noted', 'unstable', 'fracture', 'of', 'posterior_elements', 'major_surgical', 'or', 'invasive', 'procedure', 'anterior', 'cervical', 'osteotomy', 'with', 'decompression', 'and', 'excision', 'of', 'ossification', 'of', 'the', 'posterior_longitudinal_ligament', 'anterior', 'cervical', 'deformity', 'correction', 'interbody', 'reconstruction', 'anterior', 'cervical', 'fusion', 'plate', 'instrumentation', 'cervical', 'laminectomy', 'posterior', 'cervical', 'arthrodesis', 'cervical', 'instrumentation', 'arthrodesis', 'augmentation', 'with', 'autograft', 'allograft', 'and', 'demineralized', 'bone', 'matrix', 'history', 'of', 'p

In [16]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [17]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['admission', 'date', 'discharge', 'date', 'date', 'service_medicine', 'allergy', 'patient', 'record', 'know', 'allergy', 'drug', 'attend', 'first', 'name', 'lf', 'chief_complaint', 'mr', 'know', 'lastname', 'see', 'hospital', 'mechanical', 'fall', 'height', 'foot', 'scan', 'note', 'unstable', 'fracture', 'posterior', 'element', 'major_surgical', 'invasive', 'procedure', 'anterior', 'cervical', 'osteotomy', 'decompression', 'excision', 'ossification', 'posterior', 'longitudinal_ligament', 'anterior', 'cervical', 'deformity', 'correction', 'interbody', 'reconstruction', 'anterior', 'cervical', 'fusion', 'plate', 'instrumentation', 'cervical', 'laminectomy', 'posterior', 'cervical', 'arthrodesis', 'cervical', 'instrumentation', 'arthrodesis', 'augmentation', 'autograft', 'allograft', 'demineralize', 'bone', 'matrix', 'history', 'present', 'illness', 'mr', 'know', 'lastname', 'year', 'old', 'male', 'slip', 'fall', 'approximately', 'feet', 'balcony', 'ambulatory', 'scene', 'present', 'ed'

In [18]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 2), (2, 2), (3, 1), (4, 1), (5, 2), (6, 1), (7, 2), (8, 1), (9, 1), (10, 1), (11, 3), (12, 1), (13, 1), (14, 2), (15, 1), (16, 1), (17, 6), (18, 1), (19, 2), (20, 1), (21, 4), (22, 1), (23, 6), (24, 1), (25, 1), (26, 10), (27, 1), (28, 2), (29, 1), (30, 2), (31, 1), (32, 1), (33, 3), (34, 1), (35, 1), (36, 2), (37, 1), (38, 1), (39, 6), (40, 1), (41, 3), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 2), (51, 1), (52, 1), (53, 2), (54, 1), (55, 1), (56, 1), (57, 2), (58, 7), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 5), (65, 3), (66, 6), (67, 1), (68, 2), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 3), (78, 1), (79, 1), (80, 1), (81, 2), (82, 1), (83, 1), (84, 5), (85, 2), (86, 1), (87, 1), (88, 1), (89, 1), (90, 2), (91, 1), (92, 10), (93, 3), (94, 5), (95, 1), (96, 5), (97, 1), (98, 1), (99, 3), (100, 1), (101, 1), (102, 1), (103, 1), (104, 1), (105, 3), (106, 2), (107, 1), (108, 1), (109, 1), (110, 

In [19]:
# import pickle 
# filehandler = open('mimic_corpus.pickle', 'wb') 
# pickle.dump(corpus, filehandler)
# filehandler.close()

# filehandler = open('id2word.pickle', 'wb') 
# pickle.dump(id2word, filehandler)
# filehandler.close()


In [4]:
import pickle 
with open('mimic_corpus.pickle', 'rb') as inp:
    corpus = pickle.load(inp)
    
with open('id2word.pickle', 'rb') as inp:
    id2word = pickle.load(inp)

In [ ]:
from gensim.models.callbacks import CallbackAny2Vec
class EpochLogger(CallbackAny2Vec):
    '''Callback to log information about training'''
    def __init__(self):
        self.epoch = 0

    def on_epoch_begin(self, model):
        print("Epoch #{} start".format(self.epoch))

    def on_epoch_end(self, model):
        print("Epoch #{} end".format(self.epoch))
        self.epoch += 1

In [5]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=60, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [7]:
import pickle 
filehandler = open('lda_model.pickle', 'wb') 
pickle.dump(lda_model, filehandler)
filehandler.close()

In [11]:
lda_model.print_topics(60, 30)


[(0,
  '0.196*"chronic" + 0.172*"pulmonary" + 0.152*"disease" + 0.116*"demand" + 0.105*"worsen" + 0.060*"transition" + 0.055*"electrolyte" + 0.046*"primary" + 0.020*"earlier" + 0.015*"peak" + 0.011*"delay" + 0.007*"hyperkalemia" + 0.005*"normalize" + 0.000*"requiri" + 0.000*"nonequilibrium" + 0.000*"bileoma" + 0.000*"reformated" + 0.000*"thymic_hyperplasia" + 0.000*"charaterizing" + 0.000*"mastid" + 0.000*"nonionic_iodinat" + 0.000*"adavan" + 0.000*"fluorscopy" + 0.000*"immediate_postprocedure" + 0.000*"cobra_glide" + 0.000*"intraarterially" + 0.000*"contrastclip_telephone" + 0.000*"prope" + 0.000*"riolan" + 0.000*"staphoccus"'),
 (1,
  '0.176*"increase" + 0.176*"change" + 0.077*"improve" + 0.070*"date" + 0.058*"decrease" + 0.058*"control" + 0.056*"rate" + 0.046*"slightly" + 0.045*"sinus" + 0.031*"new" + 0.030*"occasional" + 0.025*"significant" + 0.025*"tachycardia" + 0.018*"otherwise" + 0.012*"frequent" + 0.011*"previous" + 0.011*"slow" + 0.011*"response" + 0.009*"persist" + 0.005*"re

In [16]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/nfs/home/ramtin/environments/experiments/lib/python3.5/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
23     12.828824        1       1 -0.223836 -0.277360
35     10.859099        1       2 -0.217096 -0.203265
13      7.708740        1       3 -0.219800 -0.171115
1       7.324284        1       4 -0.212340 -0.114608
49      6.327005        1       5 -0.206345 -0.057800
17      4.124853        1       6 -0.196243 -0.002222
44      3.967473        1       7 -0.213146 -0.102528
56      3.900053        1       8 -0.194040  0.007173
8       3.448884        1       9 -0.185811  0.034343
15      3.117360        1      10 -0.183800  0.040563
59      2.989915        1      11 -0.191384  0.019569
50      2.799469        1      12 -0.193290  0.015044
14      2.395039        1      13 -0.196660 -0.000478
9       2.319772        1      14 -0.199285 -0.007351
11      2.317918        1      15 -0.147510  0.072195
42      2.263390        1      16 -0.188938  0.029123
32      1.687896        1      17 -0.160009  0.070078
0       1.558751        1      18 -0.128563  0.072239
27      1.516496        1      19 -0.171504  0.061602
58      1.497057        1      20 -0.149388  0.074459
40      1.462486        1      21 -0.152705  0.073674
34      1.425622        1      22 -0.167588  0.065171
30      1.287966        1      23 -0.029016  0.031593
43      1.015490        1      24 -0.099989  0.063028
12      0.982162        1      25 -0.103904  0.064961
46      0.901643        1      26 -0.184565  0.040907
54      0.855579        1      27 -0.143061  0.075066
36      0.758645        1      28 -0.085719  0.056968
45      0.697893        1      29 -0.008566  0.023527
25      0.676650        1      30 -0.080647  0.054601
26      0.517376        1      31 -0.021946  0.028864
52      0.485849        1      32 -0.098703  0.062543
57      0.464232        1      33 -0.008643  0.023763
5       0.442125        1      34  0.103977 -0.006185
20      0.434076        1      35  0.010366  0.017005
2       0.374428        1      36  0.022045  0.013235
6       0.257203        1      37  0.185078 -0.014320
7       0.233633        1      38  0.077790 -0.001253
19      0.214544        1      39  0.087105 -0.003126
41      0.212442        1      40  0.092762 -0.004202
29      0.203030        1      41  0.167983 -0.013458
16      0.197377        1      42  0.151765 -0.012207
4       0.152864        1      43  0.155875 -0.012563
3       0.143442        1      44  0.168677 -0.013498
47      0.136639        1      45  0.090951 -0.003857
28      0.096545        1      46  0.187919 -0.014410
33      0.088773        1      47  0.236278 -0.013556
55      0.069992        1      48  0.243918 -0.012816
53      0.061634        1      49  0.194083 -0.014577
51      0.050233        1      50  0.203223 -0.014682
24      0.033463        1      51  0.262701 -0.009314
31      0.029032        1      52  0.245774 -0.012594
21      0.024984        1      53  0.248275 -0.012263
39      0.021867        1      54  0.247148 -0.012416
10      0.019438        1      55  0.252226 -0.011659
37      0.006882        1      56  0.263328 -0.009122
22      0.003031        1      57  0.266198 -0.008122
18      0.002945        1      58  0.266198 -0.008122
48      0.002839        1      59  0.266198 -0.008122
38      0.002667        1      60  0.266198 -0.008122, topic_info=       Category           Freq           Term          Total  loglift  logprob
53703   Default  877041.000000       continue  877041.000000  30.0000  30.0000
51285   Default  981852.000000           note  981852.000000  29.0000  29.0000
113725  Default  934558.000000           name  934558.000000  28.0000  28.0000
94618   Default  614464.000000           last  614464.000000  27.0000  27.0000
68804   Default  544588.000000            day  544588.000000  26.0000  26.0000
16171   Default  540404.000000         remain  540404.000000  25.0000  25.0000
110872  Default  940434.000000           well  940434.00

In [13]:
lda_model[corpus[0]]

([(0, 0.017146593),
  (1, 0.06070309),
  (5, 0.04637674),
  (8, 0.0533432),
  (11, 0.022598853),
  (12, 0.012903527),
  (13, 0.04199395),
  (15, 0.015145369),
  (17, 0.07731377),
  (23, 0.10657105),
  (27, 0.020948227),
  (30, 0.04014973),
  (32, 0.015756825),
  (35, 0.13275193),
  (42, 0.013379597),
  (44, 0.017990036),
  (46, 0.028880198),
  (49, 0.097077094),
  (50, 0.011624694),
  (56, 0.042431477),
  (58, 0.017547654),
  (59, 0.012986066)],
 [(0, []),
  (1, []),
  (2, []),
  (3, []),
  (4, [11]),
  (5, []),
  (6, []),
  (7, [35]),
  (8, [32]),
  (9, []),
  (10, []),
  (11, []),
  (12, []),
  (13, []),
  (14, []),
  (15, []),
  (16, []),
  (17, []),
  (18, []),
  (19, []),
  (20, []),
  (21, [35]),
  (22, []),
  (23, []),
  (24, []),
  (25, []),
  (26, [30]),
  (27, []),
  (28, [35]),
  (29, [20]),
  (30, [11]),
  (31, []),
  (32, [23]),
  (33, []),
  (34, []),
  (35, []),
  (36, []),
  (37, []),
  (38, []),
  (39, []),
  (40, []),
  (41, [35]),
  (42, []),
  (43, [23]),
  (44, [])

In [15]:
lda_model[corpus[2]]

([(0, 0.026681915),
  (1, 0.040463373),
  (8, 0.10785457),
  (9, 0.040935416),
  (13, 0.0684661),
  (14, 0.024890872),
  (15, 0.038381346),
  (17, 0.0127223255),
  (23, 0.15895776),
  (30, 0.018088311),
  (32, 0.010832652),
  (33, 0.014298756),
  (35, 0.11185737),
  (40, 0.027849298),
  (42, 0.022064619),
  (43, 0.012471414),
  (44, 0.027750703),
  (45, 0.023425253),
  (49, 0.05108951),
  (50, 0.022742316),
  (52, 0.015084109),
  (54, 0.010696218),
  (56, 0.0124140205),
  (58, 0.010191728),
  (59, 0.03008234)],
 [(1, []),
  (3, []),
  (7, [35]),
  (8, [32]),
  (11, []),
  (12, []),
  (19, []),
  (21, [35]),
  (26, [30]),
  (28, [35]),
  (30, [11]),
  (32, [23]),
  (59, []),
  (63, []),
  (68, []),
  (84, []),
  (87, []),
  (94, [1]),
  (96, [11]),
  (98, []),
  (99, [0]),
  (102, [50]),
  (105, []),
  (111, []),
  (126, [44]),
  (132, [23]),
  (137, []),
  (139, []),
  (140, []),
  (145, [1]),
  (146, [9]),
  (152, [33]),
  (171, []),
  (175, [0]),
  (186, [34]),
  (188, [35]),
  (191,